In [7]:
import re
from Tokenization.tokenization import CharacterTokenization

# Load file
vocab_file = r"C:\Users\91978\Desktop\GPT-2-Scratch\dataset\Poems.txt"
with open(vocab_file, "r", encoding="utf-8") as f:
    text = f.read()

print("Original text length:", len(text))

text = re.sub(r'\d+', '', text)
#remove special characters
text = re.sub(r'[^\w\s]', '', text)
#remove greek characters
text = re.sub(r'[α-ωΑ-Ω]', '', text)

#'²', '³', '¹', '¾', 'À', 'Á', 'Ä', 'Æ remove similar characters

print("Cleaned text length:", len(text))

# Create vocabulary from cleaned text
vocab = sorted(list(set(text)))
print("Vocabulary size:", len(vocab))
print("First 10 characters:", vocab[:10])
print(vocab)
# Test tokenization
tokenizer = CharacterTokenization(vocab)
test_text = "Hello World"
tokens = tokenizer.tokenize(test_text)
print("Tokens:", tokens)
decoded=tokenizer.detokenize(tokens)
print("Decoded:", decoded)

Original text length: 35370769
Cleaned text length: 33908384
Vocabulary size: 111
First 10 characters: ['\t', '\n', ' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
['\t', '\n', ' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '²', '³', '¹', '¾', 'À', 'Á', 'Ä', 'Æ', 'È', 'É', 'Ñ', 'Ô', 'Õ', 'Ü', 'Ý', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ù', 'ú', 'û', 'ü', 'ý', 'ÿ', 'Œ', 'œ', 'š', 'Ž', 'ž', 'ל', 'מ', 'ת', '饟']
Tokens: [10, 34, 41, 41, 44, 2, 25, 44, 47, 41, 33]
Decoded: Hello World


In [ ]:
text = re.sub(r'\d+', '', text)
#remove special characters
text = re.sub(r'[^\w\s]', '', text)
#remove greek characters
text = re.sub(r'[α-ωΑ-Ω]', '', text)